In [ ]:
!pip install ffmpeg-python


In [ ]:
import subprocess
import os

def extract_audio(video_path, output_audio_path):
    """
    Extracts audio from a video file using ffmpeg.
    """

    if not os.path.exists(video_path):
        raise FileNotFoundError("Video file not found")

    command = [
        "ffmpeg",
        "-i", video_path,
        "-vn",                  # Disable video
        "-acodec", "mp3",       # Audio codec
        "-ab", "192k",          # Audio bitrate
        output_audio_path,
        "-y"                    # Overwrite if exists
    ]

    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("Audio extracted successfully:", output_audio_path)


# Example usage
extract_audio("sample_video.mp4", "extracted_audio.mp3")


Audio extracted successfully: extracted_audio.mp3


In [ ]:
! pip install openai-whisper soundfile numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=9131d4b9c8b0a1163709a886cc4c1b13ee592b517292a8b5d85bf6f782a627d3
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [ ]:
import whisper
import numpy as np
from collections import defaultdict

model = whisper.load_model("small")
audio_path = "extracted_audio.mp3"

base_result = model.transcribe(
    audio_path,
    task="transcribe",
    verbose=False
)

audio = whisper.load_audio(audio_path)
sample_rate = 16000

def detect_language_for_segment(audio, start, end, model, sr=16000):
    segment_audio = audio[int(start * sr): int(end * sr)]

    # Skip extremely short segments
    if len(segment_audio) < sr * 1.0:  # < 1 second
        return "unknown", 0.0

    # 🔑 CRITICAL FIX: pad or trim to 30 seconds
    segment_audio = whisper.pad_or_trim(segment_audio)

    mel = whisper.log_mel_spectrogram(segment_audio).to(model.device)
    _, probs = model.detect_language(mel)

    lang = max(probs, key=probs.get)
    confidence = probs[lang]

    return lang, confidence

languages_detected = defaultdict(list)

for seg in base_result["segments"]:
    start, end, text = seg["start"], seg["end"], seg["text"].strip()

    if len(text.split()) < 3:
        continue

    lang, conf = detect_language_for_segment(audio, start, end, model)

    if conf < 0.70:
        lang = "unknown"

    languages_detected[lang].append({
        "start": round(start, 2),
        "end": round(end, 2),
        "confidence": round(conf, 2),
        "text": text
    })

print("\nDetected Languages:\n")

for lang, segments in languages_detected.items():
    print(f"\nLanguage: {lang}")
    for s in segments:
        print(f"[{s['start']}s - {s['end']}s] ({s['confidence']}) {s['text']}")


100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 104MiB/s]


Detected language: English


100%|██████████| 3682/3682 [00:04<00:00, 884.25frames/s]



Detected Languages:


Language: en
[0.0s - 3.04s] (1.0) If you still haven't heard about this AI tool, you need to check it out.
[3.04s - 6.0s] (0.97) Okay, I'm speaking English to you right now, right?
[17.76s - 21.64s] (0.99) The crazy part is that this whole time, I'm just speaking English,
[21.64s - 25.64s] (0.99) and all I did was uploading this video to Heijan's AI translator,
[25.64s - 28.88s] (1.0) and it cloned my voice into different languages that you just heard.
[28.92s - 32.96s] (0.99) If that wasn't impressive enough, it also synced my lips to the new voice
[32.96s - 36.8s] (1.0) so that the final clips looks and sounds as authentic as possible.

Language: hi
[6.0s - 8.76s] (0.99) L'écénagre me baton qui met habibi anglésie-boul-ranhon,

Language: fr
[10.0s - 12.36s] (0.97) laissez-moi passer en français un instant.
[12.36s - 13.92s] (0.94) Mon français sonne comment?

Language: de
[13.92s - 15.72s] (0.97) Au avis vert, it's me Deutsch.
[15.72s - 17.76s] (0.88) Clink my 

In [ ]:
from groq import Groq
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

client = Groq(api_key="")

def record(filename="audio.wav"):
    print("Recording...")
    duration = 5
    samplerate = 16000

    audio = sd.rec(int(duration * samplerate), samplerate=samplerate,
                   channels=1, dtype='int16')
    sd.wait()

    wav.write(filename, samplerate, audio)
    return filename

while True:
    file_path = record()

    response = client.audio.transcriptions.create(
        file=open(file_path, "rb"),
        model="whisper-large-v3",
        response_format="verbose_json"
    )

    print("Language:", response.language)
    print("Text:", response.text)
    print("-" * 50)

In [ ]:
# Install required packages
!pip install openai-whisper langdetect pydub
!apt-get install ffmpeg

import whisper
from langdetect import detect, LangDetectException
from pydub import AudioSegment

# --------------------------------------------------
# STEP 1: Convert MP3 to WAV
# --------------------------------------------------

audio_path = "/content/pushpa_dialogue.mp3"
wav_path = "/content/pushpa_dialogue.wav"

AudioSegment.from_mp3(audio_path).export(wav_path, format="wav")
print("MP3 converted to WAV")

# --------------------------------------------------
# STEP 2: Load Whisper model
# --------------------------------------------------

print("Loading Whisper model... (takes 20–30 sec)")
model = whisper.load_model("small")   # You can use "base" if GPU is slow

# --------------------------------------------------
# STEP 3: Transcribe audio
# --------------------------------------------------

print("Transcribing...")
result = model.transcribe(wav_path)
text = result["text"].strip()

print("\nTranscribed Text:\n", text)

# --------------------------------------------------
# STEP 4: Detect language from text
# --------------------------------------------------

try:
    lang = detect(text)
    print("\nDetected Language Code:", lang)
except LangDetectException:
    print("Error: Could not detect language (text is too short)")